In [22]:
import pandas             as pd
import numpy              as np
import matplotlib.pyplot  as plt
from plyer import notification 
from datetime import datetime
import requests
import os
import zipfile

In [24]:
def baixar_e_extrair(url, nome_arquivo):
  
  # Verifica se o arquivo já existe
  if os.path.exists(nome_arquivo):
    print(f"O arquivo {nome_arquivo} já existe. Pulando o download.")
    return

  # Baixa o arquivo
  response = requests.get(url)
  response.raise_for_status()

  # Salva o arquivo no disco
  with open(nome_arquivo, 'wb') as f:
    f.write(response.content)

  # Extrai o arquivo ZIP
  with zipfile.ZipFile(nome_arquivo, 'r') as zip_ref:
    zip_ref.extractall()

# URL do arquivo ZIP:
url = "https://download.inep.gov.br/microdados/microdados_enem_2023.zip"  
nome_arquivo = "microdados_enem_2023.zip"  # Nome do arquivo baixado

# Chama a função para baixar e extrair o arquivo
baixar_e_extrair(url, nome_arquivo)

O arquivo microdados_enem_2023.zip já existe. Pulando o download.


In [25]:
def alerta(nivel, base, etapa, erro=""):
    now = str(datetime.now())
    msg = f"Falha no carregamento da base {base} na etapa {etapa}.\n{now}\n{erro}"

    if nivel == 1:
        title = 'ATENÇÃO: Alerta Baixo'
    elif nivel == 2:
        title = 'ATENÇÃO: Alerta Médio'
    elif nivel == 3:
        title = 'ATENÇÃO: Alerta Alto'
    else:
        print("Nível", nivel, "não disponível!")
        return

    # Enviar notificação
    notification.notify(
        title=title,
        message=msg,
        app_name='alerta',
        timeout=10
    )

In [ ]:
def alerta(nivel, base, etapa, erro=""):
    now = str(datetime.now())
    msg = f"Falha no carregamento da base {base} na etapa {etapa}.\n{now}\n{erro}"

    if nivel == 1:
        title = 'ATENÇÃO: Alerta Baixo'
    elif nivel == 2:
        title = 'ATENÇÃO: Alerta Médio'
    elif nivel == 3:
        title = 'ATENÇÃO: Alerta Alto'
    else:
        print("Nível", nivel, "não disponível!")
        return

    # Enviar notificação
    notification.notify(
        title=title,
        message=msg,
        app_name='alerta',
        timeout=10
    )

# Função para carregar CSV
def carregar_csv(caminho_csv):
    try:
        return pd.read_csv(caminho_csv, encoding='latin1', sep=';')
    except:
        return None

# Caminho para o arquivo CSV
caminho_csv = "DADOS/MICRODADOS_ENEM_2023.csv"

# Carrega o arquivo em um DataFrame
df = carregar_csv(caminho_csv)

# Verifica se o DataFrame foi carregado com sucesso
if df is None:
    alerta(3, "MICRODADOS_ENEM_2023.csv", "Leitura do CSV", "Erro ao carregar o arquivo")
else:
    print(df.head(5))

In [ ]:
df.info

<bound method DataFrame.info of          NU_INSCRICAO  NU_ANO  TP_FAIXA_ETARIA TP_SEXO  TP_ESTADO_CIVIL  \
0        210059085136    2023               14       M                2   
1        210059527735    2023               12       M                2   
2        210061103945    2023                6       F                1   
3        210060214087    2023                2       F                1   
4        210059980948    2023                3       F                1   
...               ...     ...              ...     ...              ...   
3933950  210061959676    2023               12       M                1   
3933951  210061950911    2023                1       F                1   
3933952  210061965966    2023                3       F                1   
3933953  210061932304    2023                2       M                1   
3933954  210058924455    2023                3       F                1   

         TP_COR_RACA  TP_NACIONALIDADE  TP_ST_CONCLUSAO  TP_ANO_CON

In [ ]:
df.shape

(3933955, 76)

In [ ]:
df.columns

Index(['NU_INSCRICAO', 'NU_ANO', 'TP_FAIXA_ETARIA', 'TP_SEXO',
       'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO',
       'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO',
       'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC',
       'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC',
       'CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA', 'CO_UF_PROVA',
       'SG_UF_PROVA', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
       'TP_PRESENCA_MT', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC',
       'CO_PROVA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
       'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC',
       'TX_RESPOSTAS_MT', 'TP_LINGUA', 'TX_GABARITO_CN', 'TX_GABARITO_CH',
       'TX_GABARITO_LC', 'TX_GABARITO_MT', 'TP_STATUS_REDACAO',
       'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4',
       'NU_NOTA_COMP5', 'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004',

In [ ]:
dados_ausentes = df.isna().sum()
dados_ausentes

NU_INSCRICAO       0
NU_ANO             0
TP_FAIXA_ETARIA    0
TP_SEXO            0
TP_ESTADO_CIVIL    0
                  ..
Q021               0
Q022               0
Q023               0
Q024               0
Q025               0
Length: 76, dtype: int64

In [ ]:
def calcular_percentual_missing(df):
    # Calculando o percentual de valores ausentes
    percentual_missing = df.isna().sum() / len(df) * 100
    
    # Ordenando os resultados em ordem decrescente
    percentual_missing = percentual_missing.sort_values(ascending=False)
    
    return percentual_missing

In [ ]:
calcular_percentual_missing(df)

TP_LOCALIZACAO_ESC        75.635054
CO_UF_ESC                 75.635054
TP_SIT_FUNC_ESC           75.635054
TP_DEPENDENCIA_ADM_ESC    75.635054
SG_UF_ESC                 75.635054
                            ...    
TP_PRESENCA_LC             0.000000
TP_PRESENCA_MT             0.000000
NU_ANO                     0.000000
TP_LINGUA                  0.000000
Q025                       0.000000
Length: 76, dtype: float64

In [ ]:
df.index

RangeIndex(start=0, stop=3933955, step=1)

In [ ]:
df.nunique().sort_values(ascending=True)

NU_ANO                   1
Q025                     2
Q021                     2
Q020                     2
Q018                     2
                    ...   
TX_RESPOSTAS_MT    2673883
TX_RESPOSTAS_CN    2675115
TX_RESPOSTAS_CH    2814295
TX_RESPOSTAS_LC    2819954
NU_INSCRICAO       3933955
Length: 76, dtype: int64

In [ ]:
type(df)

pandas.core.frame.DataFrame

In [ ]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
NU_INSCRICAO,3933955.0,2.100601e+11,1.154550e+06,2.100580e+11,2.100591e+11,2.100601e+11,2.100611e+11,2.100621e+11
NU_ANO,3933955.0,2.023000e+03,0.000000e+00,2.023000e+03,2.023000e+03,2.023000e+03,2.023000e+03,2.023000e+03
TP_FAIXA_ETARIA,3933955.0,5.096262e+00,3.871855e+00,1.000000e+00,2.000000e+00,3.000000e+00,7.000000e+00,2.000000e+01
TP_ESTADO_CIVIL,3933955.0,1.043938e+00,4.120029e-01,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,4.000000e+00
TP_COR_RACA,3933955.0,2.058200e+00,1.003314e+00,0.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00,5.000000e+00
TP_NACIONALIDADE,3933955.0,1.028277e+00,2.127761e-01,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,4.000000e+00
TP_ST_CONCLUSAO,3933955.0,1.684697e+00,7.467924e-01,1.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00,4.000000e+00
TP_ANO_CONCLUIU,3933955.0,2.445029e+00,4.450337e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,1.700000e+01
TP_ESCOLA,3933955.0,1.415810e+00,6.018231e-01,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00
TP_ENSINO,1339081.0,1.005142e+00,7.152547e-02,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00


In [ ]:
def encontrar_dataframe_matematica(df):
    # Filtrar as colunas desejadas
    df_matematica = df[['TP_FAIXA_ETARIA', 'TP_SEXO', 'CO_PROVA_MT']]
    
    # Remover linhas com valores NaN
    df_matematica_limpa = df_matematica.dropna()
    
    return df_matematica_limpa

In [ ]:
encontrar_dataframe_matematica(df)

,TP_FAIXA_ETARIA,TP_SEXO,CO_PROVA_MT
2,6,F,1211.0
3,2,F,1214.0
4,3,F,1212.0
9,11,M,1213.0
10,8,M,1211.0
...,...,...,...
3933946,12,F,1212.0
3933948,3,M,1214.0
3933950,12,M,1211.0
3933951,1,F,1211.0


In [ ]:
df_matematica_limpa = encontrar_dataframe_matematica(df)

In [ ]:
df_matematica_limpa.shape

(2692427, 3)

In [ ]:
df_matematica_limpa.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
TP_FAIXA_ETARIA,2692427.0,4.390505,3.496942,1.0,2.0,3.0,5.0,20.0
CO_PROVA_MT,2692427.0,1212.664779,3.691568,1211.0,1212.0,1213.0,1214.0,1294.0


In [ ]:
df_matematica_limpa.TP_SEXO.value_counts()

TP_SEXO
F    1651485
M    1040942
Name: count, dtype: int64